In [66]:
# =======================
# Cell 1: Imports & Config
# =======================
import os
import time
import random
import warnings
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score,
    f1_score, precision_score, confusion_matrix
)
import matplotlib.pyplot as plt
import seaborn as sns

# Import custom model & configs
import sys
sys.path.append("../Utils")
from models import DGCNN  # DGCNN & PointNet implementation
import configs

# Paths
TRAIN_DIR = Path(configs.TRAIN_DIR)
TEST_DIR = Path(configs.TEST_DIR)
MODEL_DIR = Path(configs.MODEL_DIR)
MODEL_DIR.mkdir(exist_ok=True)

# Force CPU device (no CUDA)
class CFG:
    num_points = 1024
    batch_size = 16
    epochs = 20
    lr = 1e-3
    device = torch.device("cpu")  # ✅ FORCE CPU
    k = 20
    emb_dims = 1024
    dropout = 0.5
    num_classes = len(os.listdir(TRAIN_DIR))

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

print("⚠️ Running on CPU (no CUDA used)")


⚠️ Running on CPU (no CUDA used)


In [67]:
# =======================
# Cell 2: Dataset & Utils
# =======================
def load_point_file_safe(file_path):
    """Load a point cloud file robustly with multiple delimiters."""
    for delim in [None, " ", ",", "\t"]:
        try:
            pc = np.loadtxt(file_path, delimiter=delim)
            if pc.ndim == 1:
                pc = pc[np.newaxis, :]
            return pc
        except Exception:
            continue
    raise ValueError(f"Could not read {file_path}")

def normalize_unit(pc):
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    return pc / m

def farthest_point_sampling(pc, n_points):
    N, _ = pc.shape
    if N <= n_points:
        return np.pad(pc, ((0, n_points - N), (0, 0)))
    centroids = np.zeros((n_points,))
    distance = np.ones(N) * 1e10
    farthest = np.random.randint(0, N)
    for i in range(n_points):
        centroids[i] = farthest
        dist = np.sum((pc - pc[farthest])**2, axis=1)
        distance = np.minimum(distance, dist)
        farthest = np.argmax(distance)
    return pc[centroids.astype(np.int32)]

class PointCloudDataset(Dataset):
    def __init__(self, root_dir, num_points=1024):
        self.root_dir = Path(root_dir)
        self.num_points = num_points
        self.files, self.labels = [], []
        for i, class_dir in enumerate(sorted(self.root_dir.iterdir())):
            if class_dir.is_dir():
                for f in class_dir.glob("*.*"):
                    self.files.append(f)
                    self.labels.append(i)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        try:
            pc = load_point_file_safe(self.files[idx])
            if pc.shape[1] > 3:
                pc = pc[:, :3]
            pc = normalize_unit(pc)
            pc = farthest_point_sampling(pc, self.num_points)
            return torch.from_numpy(pc.T.astype(np.float32)), int(self.labels[idx])
        except Exception as e:
            warnings.warn(f"⚠️ Error loading idx={idx}: {e}")
            return torch.zeros((3, self.num_points), dtype=torch.float32), 0


In [68]:
# =======================
# Cell 3: DataLoaders
# =======================
train_dataset = PointCloudDataset(TRAIN_DIR, num_points=CFG.num_points)
test_dataset = PointCloudDataset(TEST_DIR, num_points=CFG.num_points)

val_size = int(0.1 * len(train_dataset))
train_size = len(train_dataset) - val_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

# ✅ pin_memory removed (no CUDA)
train_loader = DataLoader(train_subset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

print(f"Training samples:   {len(train_subset)}")
print(f"Validation samples: {len(val_subset)}")
print(f"Test samples:       {len(test_dataset)}")


Training samples:   502
Validation samples: 55
Test samples:       134


In [69]:
# =======================
# Cell 4: Model Setup
# =======================
model = DGCNN(CFG, output_channels=CFG.num_classes).to(CFG.device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=CFG.lr)

print(f"Device: {CFG.device}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")


Device: cpu
Model parameters: 1,801,095


In [70]:
# =======================
# Cell 5: Train/Eval Functions
# =======================
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss, total_correct, total_samples = 0, 0, 0
    for points, labels in loader:
        points, labels = points.to(device), labels.to(device)
        optimizer.zero_grad()
        preds = model(points)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * points.size(0)
        total_correct += preds.argmax(1).eq(labels).sum().item()
        total_samples += labels.size(0)
    return total_loss / total_samples, total_correct / total_samples

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct, total_samples = 0, 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        for points, labels in loader:
            points, labels = points.to(device), labels.to(device)
            preds = model(points)
            loss = criterion(preds, labels)
            total_loss += loss.item() * points.size(0)
            total_correct += preds.argmax(1).eq(labels).sum().item()
            total_samples += labels.size(0)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.argmax(1).cpu().numpy())
    return total_loss / total_samples, total_correct / total_samples, np.array(all_labels), np.array(all_preds)


In [71]:
# =======================
# Cell 6: Training Loop
# =======================
best_val_acc = 0
train_losses, val_losses = [], []
train_accs, val_accs = [], []
start_time = time.time()
patience, patience_counter = 5, 0

for epoch in range(CFG.epochs):
    epoch_start = time.time()

    tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, criterion, CFG.device)
    val_loss, val_acc, val_labels, val_preds = evaluate(model, val_loader, criterion, CFG.device)

    train_losses.append(tr_loss)
    train_accs.append(tr_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "val_acc": val_acc
        }, MODEL_DIR / "best_model.pth")
        print(f"✓ New best model saved (Val Acc: {val_acc:.4f})")
    else:
        patience_counter += 1

    print(f"Epoch [{epoch+1}/{CFG.epochs}] | "
          f"Train Loss: {tr_loss:.4f} | Train Acc: {tr_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | "
          f"Epoch Time: {(time.time()-epoch_start):.1f}s")

    if patience_counter >= patience:
        print("⚠️ Early stopping triggered.")
        break

exec_time = time.time() - start_time
print(f"\n✅ Training finished in {exec_time:.1f}s | Best Val Acc: {best_val_acc:.4f}")

test_loss, test_acc, test_labels, test_preds = evaluate(model, test_loader, criterion, CFG.device)
print(f"Final Test Accuracy: {test_acc:.4f}")


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# =======================
# Cell 7: Metrics & Plots
# =======================
overall_acc = accuracy_score(val_labels, val_preds)
balanced_acc = balanced_accuracy_score(val_labels, val_preds)
f1 = f1_score(val_labels, val_preds, average="weighted")
prec_per_class = precision_score(val_labels, val_preds, average=None)

print("📊 Validation Metrics:")
print(f"Accuracy:          {overall_acc:.4f}")
print(f"Balanced Accuracy: {balanced_acc:.4f}")
print(f"F1 Score:          {f1:.4f}")
print(f"Precision/Class:   {prec_per_class}")

cm = confusion_matrix(val_labels, val_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.legend(); plt.title("Loss Curve")

plt.subplot(1, 2, 2)
plt.plot(train_accs, label="Train Acc")
plt.plot(val_accs, label="Val Acc")
plt.legend(); plt.title("Accuracy Curve")
plt.show()
